In [1]:
import marimo as mo

In [2]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, roc_auc_score
import matplotlib.pyplot as plt
import yaml
import warnings
import optuna
from optuna.visualization import plot_optimization_history, plot_param_importances

warnings.filterwarnings("ignore")

/Users/shosei/workspace/kaggle/competitions/table/league_of_legends_winning_prediction/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# データ準備
X_train = pd.read_csv("../data/processed/train_x_add_features.csv")
test_df = pd.read_csv("../data/processed/test_x_add_features.csv")

y_train = X_train["win"]
X_train.drop(["win"], axis=1, inplace=True)
X_test = test_df.drop(["id", "win"], axis=1)

print(X_train.shape, X_test.shape, y_train.shape, test_df.shape)

(144000, 42) (36000, 42) (144000,) (36000, 44)


In [5]:
config_path = "../configs/config_lgbm_cvmean.yaml"
with open(config_path) as config_file:
    config = yaml.safe_load(config_file)

# 交差検証の設定
n_splits = 5
optimal_threshold = config["threshold"]["optimal"]
kf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=71)

len(X_train) // n_splits

28800

In [6]:
# Optunaのobjective関数を定義
def objective(trial):
    # ハイパーパラメータの探索範囲を定義
    params = {
        "objective": "binary",
        "metric": "binary_logloss",
        "boosting_type": "gbdt",
        "verbosity": -1,
        "random_state": 42,
        # チューニング対象のパラメータ
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.1, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 20, 100),
        "max_depth": trial.suggest_int("max_depth", 3, 12),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.6, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.6, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 10.0, log=True),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 10.0, log=True),
    }

    # 交差検証でスコアを計算
    cv_scores = []
    kf_optuna = StratifiedKFold(n_splits=5, shuffle=True, random_state=71)

    for train_idx, valid_idx in kf_optuna.split(X_train, y_train):
        X_train_fold = X_train.iloc[train_idx]
        X_valid_fold = X_train.iloc[valid_idx]
        y_train_fold = y_train.iloc[train_idx]
        y_valid_fold = y_train.iloc[valid_idx]

        train_data = lgb.Dataset(X_train_fold, label=y_train_fold)
        valid_data = lgb.Dataset(X_valid_fold, label=y_valid_fold, reference=train_data)

        model = lgb.train(
            params,
            train_data,
            num_boost_round=1000,
            valid_sets=[valid_data],
            callbacks=[
                lgb.early_stopping(stopping_rounds=50, verbose=False),
                lgb.log_evaluation(period=0),
            ],
        )

        y_valid_pred = model.predict(X_valid_fold, num_iteration=model.best_iteration)
        auc = roc_auc_score(y_valid_fold, y_valid_pred) # type: ignore
        cv_scores.append(auc)

    return np.mean(cv_scores)

In [7]:
# Optunaでハイパーパラメータ最適化を実行
study = optuna.create_study(direction="maximize", study_name="lgbm_optimization")
study.optimize(objective, n_trials=100, show_progress_bar=True) # type: ignore

print(f"\nBest trial:")
print(f"  Value (AUC): {study.best_trial.value:.4f}")
print(f"  Params:")
for key, value in study.best_trial.params.items():
    print(f"    {key}: {value}")

[I 2025-10-21 02:23:54,964] A new study created in memory with name: lgbm_optimization
Best trial: 0. Best value: 0.938243:   1%|          | 1/100 [02:18<3:48:35, 138.54s/it]

[I 2025-10-21 02:26:13,524] Trial 0 finished with value: 0.9382426916340387 and parameters: {'learning_rate': 0.016950642952172298, 'num_leaves': 86, 'max_depth': 7, 'min_child_samples': 64, 'feature_fraction': 0.8939086855724778, 'bagging_fraction': 0.8470745151537922, 'bagging_freq': 7, 'lambda_l1': 0.0005568936859938009, 'lambda_l2': 1.8710169838375191e-06}. Best is trial 0 with value: 0.9382426916340387.


Best trial: 0. Best value: 0.938243:   2%|▏         | 2/100 [03:11<2:23:37, 87.93s/it] 

[I 2025-10-21 02:27:06,029] Trial 1 finished with value: 0.9357287932502831 and parameters: {'learning_rate': 0.022308411991754488, 'num_leaves': 91, 'max_depth': 4, 'min_child_samples': 54, 'feature_fraction': 0.6441557087586606, 'bagging_fraction': 0.8498122787608526, 'bagging_freq': 7, 'lambda_l1': 2.0484323970347587e-06, 'lambda_l2': 2.196300731116674e-05}. Best is trial 0 with value: 0.9382426916340387.


Best trial: 0. Best value: 0.938243:   3%|▎         | 3/100 [03:53<1:48:27, 67.09s/it]

...

[I 2025-10-21 06:16:02,739] Trial 97 finished with value: 0.9388625625973699 and parameters: {'learning_rate': 0.01983195876900964, 'num_leaves': 88, 'max_depth': 10, 'min_child_samples': 95, 'feature_fraction': 0.8258266718172957, 'bagging_fraction': 0.6235701017489442, 'bagging_freq': 4, 'lambda_l1': 0.8610316512677944, 'lambda_l2': 6.304407099852359e-05}. Best is trial 54 with value: 0.9391817419040718.


Best trial: 98. Best value: 0.939183:  99%|█████████▉| 99/100 [3:54:00<01:39, 99.34s/it]

[I 2025-10-21 06:17:55,498] Trial 98 finished with value: 0.939182892075881 and parameters: {'learning_rate': 0.018755187732905482, 'num_leaves': 93, 'max_depth': 10, 'min_child_samples': 88, 'feature_fraction': 0.6229785450447526, 'bagging_fraction': 0.8425187268765811, 'bagging_freq': 4, 'lambda_l1': 3.44684901676459, 'lambda_l2': 0.0007365075504023804}. Best is trial 98 with value: 0.939182892075881.


Best trial: 98. Best value: 0.939183: 100%|██████████| 100/100 [3:55:46<00:00, 141.47s/it]

[I 2025-10-21 06:19:41,572] Trial 99 finished with value: 0.939072122847325 and parameters: {'learning_rate': 0.022695646424846654, 'num_leaves': 93, 'max_depth': 10, 'min_child_samples': 88, 'feature_fraction': 0.9425514880990513, 'bagging_fraction': 0.8380277229980803, 'bagging_freq': 4, 'lambda_l1': 3.6907769566102173, 'lambda_l2': 0.0008193813536866141}. Best is trial 98 with value: 0.939182892075881.

Best trial:
  Value (AUC): 0.9392
  Params:
    learning_rate: 0.018755187732905482
    num_leaves: 93
    max_depth: 10
    min_child_samples: 88
    feature_fraction: 0.6229785450447526
    bagging_fraction: 0.8425187268765811
    bagging_freq: 4
    lambda_l1: 3.44684901676459
    lambda_l2: 0.0007365075504023804


In [8]:
# 最適化の履歴を可視化
fig1 = plot_optimization_history(study)
fig1.show()

# パラメータの重要度を可視化
fig2 = plot_param_importances(study)
fig2.show()

In [9]:
# 最適なパラメータで再学習
best_params = study.best_trial.params
best_params.update(
    {
        "objective": "binary",
        "metric": "binary_logloss",
        "boosting_type": "gbdt",
        "verbosity": -1,
        "random_state": 42,
    }
)

# 交差検証で最終評価
n_splits_final = 5
kf_final = StratifiedKFold(n_splits=n_splits_final, shuffle=True, random_state=71)

cv_valid_auc_best = []
cv_valid_acc_best = []
oof_predictions_best = np.zeros(X_train.shape[0])
test_predictions_best = np.zeros(X_test.shape[0])

for fold, (train_idx, valid_idx) in enumerate(kf_final.split(X_train, y_train), 1):
    print(f"Final Training - Fold {fold} / {n_splits_final}")

    X_train_fold = X_train.iloc[train_idx]
    X_valid_fold = X_train.iloc[valid_idx]
    y_train_fold = y_train.iloc[train_idx]
    y_valid_fold = y_train.iloc[valid_idx]

    train_data = lgb.Dataset(X_train_fold, label=y_train_fold)
    valid_data = lgb.Dataset(X_valid_fold, label=y_valid_fold, reference=train_data)

    model = lgb.train(
        best_params,
        train_data,
        num_boost_round=1000,
        valid_sets=[train_data, valid_data],
        callbacks=[
            lgb.early_stopping(stopping_rounds=50, verbose=False),
            lgb.log_evaluation(period=0),
        ],
    )

    y_valid_pred = model.predict(X_valid_fold, num_iteration=model.best_iteration)
    oof_predictions_best[valid_idx] = y_valid_pred  # type: ignore

    test_predictions_best += model.predict(X_test, num_iteration=model.best_iteration) / n_splits_final  # type: ignore

    valid_auc = roc_auc_score(y_valid_fold, y_valid_pred)  # type: ignore
    valid_acc = accuracy_score(y_valid_fold, (y_valid_pred > optimal_threshold).astype(int))  # type: ignore

    cv_valid_auc_best.append(valid_auc)
    cv_valid_acc_best.append(valid_acc)

    print(f"Valid Accuracy: {valid_acc:.4f} | AUC: {valid_auc:.4f}")
    print(f"Best iteration: {model.best_iteration}\n")

# OOF全体の評価
oof_auc_best = roc_auc_score(y_train, oof_predictions_best)  # type: ignore
oof_acc_best = accuracy_score(y_train, (oof_predictions_best > optimal_threshold).astype(int))  # type: ignore

print(f"\n{'='*50}")
print(f"Final Results with Best Parameters:")
print(f"{'='*50}")
print(f"Valid AUC:      {np.mean(cv_valid_auc_best):.4f} ± {np.std(cv_valid_auc_best):.4f}")
print(f"Valid Accuracy: {np.mean(cv_valid_acc_best):.4f} ± {np.std(cv_valid_acc_best):.4f}")
print(f"OOF AUC:        {oof_auc_best:.4f}")
print(f"OOF Accuracy:   {oof_acc_best:.4f}")

Final Training - Fold 1 / 5
Valid Accuracy: 0.8586 | AUC: 0.9392
Best iteration: 972

Final Training - Fold 2 / 5
Valid Accuracy: 0.8548 | AUC: 0.9380
Best iteration: 991

Final Training - Fold 3 / 5
Valid Accuracy: 0.8556 | AUC: 0.9379
Best iteration: 973

Final Training - Fold 4 / 5
Valid Accuracy: 0.8582 | AUC: 0.9397
Best iteration: 995

Final Training - Fold 5 / 5
Valid Accuracy: 0.8594 | AUC: 0.9411
Best iteration: 997


Final Results with Best Parameters:
Valid AUC:      0.9392 ± 0.0012
Valid Accuracy: 0.8573 ± 0.0018
OOF AUC:        0.9392
OOF Accuracy:   0.8573


In [10]:
# 最適化されたモデルでの提出ファイル作成
y_test_pred_best = (test_predictions_best > optimal_threshold).astype(int)

save_path_optuna = "../outputs/submissions/submission_lgbm_optuna.csv"
submission_optuna = pd.DataFrame({"id": test_df["id"], "win": y_test_pred_best})
submission_optuna.to_csv(save_path_optuna, index=False)
print(f"Submission file saved to: {save_path_optuna}")

Submission file saved to: ../outputs/submissions/submission_lgbm_optuna.csv


In [11]:
# ベストパラメータを保存
best_params_to_save = {
    "model": {
        "objective": "binary",
        "metric": "binary_logloss",
        "boosting_type": "gbdt",
        "verbosity": -1,
        "random_state": 42,
        **study.best_trial.params,
    },
    "optimization": {
        "best_auc": float(study.best_trial.value), # type: ignore
        "n_trials": len(study.trials),
    },
}

save_path = "../configs/config_lgbm_optimized.yaml"
with open(save_path, "w") as config_out:
    yaml.dump(best_params_to_save, config_out, default_flow_style=False, sort_keys=False)